# Global AI Emissions Analysis

--- 
### Importing libraries and Loading the datasets

In [ ]:
import pandas as pd

df_iea = pd.read_excel('C:/Users/91904/Downloads/IEA_Energy_and_AI.xlsx')
df_energy = pd.read_csv('C:/Users/91904/Downloads/owid-energy-data.csv')

---
## IEA (International Energy Agency) Dataset of Electricity Consumption

In [2]:
print("IEA dataset shape:", df_iea.shape)
df_iea = df_iea.iloc[22:,2:] # Filtering out useless Rows & Columns
print("\nFirst few rows of IEA data:")
df_iea = df_iea.drop(['Unnamed: 6','Unnamed: 9','Unnamed: 12','Unnamed: 15'], axis=1) # Dropping NaN Columns
df_iea.columns = ['Electricity consumption (TWh)','2020','2023','2024','Base_2030','Base_2035','LiftOff_2030','LiftOff_2035','HighEff_2030','HighEff_2035','Headwinds_2030','Headwinds_2035']
df_iea.set_index('Electricity consumption (TWh)', inplace=True)
df_iea.head()

IEA dataset shape: (31, 18)

First few rows of IEA data:


,2020,2023,2024,Base_2030,Base_2035,LiftOff_2030,LiftOff_2035,HighEff_2030,HighEff_2035,Headwinds_2030,Headwinds_2035
Electricity consumption (TWh),,,,,,,,,,,
Total,269.0,361.0,416.0,946,1193,1264,1719,792,972,669,707
Hyperscale,100.0,148.0,166.0,378,466,479,626,397,472,279,293
Colocation and service provider,85.0,112.0,144.0,355,493,482,721,385,490,246,285
Enterprise,85.0,100.0,106.0,213,234,303,372,10,10,144,128
IT,176.0,252.0,295.0,733,985,972,1409,657,864,522,587


---
## OWID (Our World In Data) Energy Dataset

In [3]:
print("OWID Energy dataset shape:", df_energy.shape)
columns_to_display = ['country','year','carbon_intensity_elec'] 
condition = ((df_energy['year'] == 2020)|(df_energy['year'] == 2023)) # Only values for Years 2020 and 2023
df_carbon_intensity = df_energy.loc[condition, columns_to_display] # Filtering out unrequired columns
df_carbon_intensity

OWID Energy dataset shape: (23195, 130)


,country,year,carbon_intensity_elec
20,ASEAN (Ember),2020,589.474
23,ASEAN (Ember),2023,570.005
145,Afghanistan,2020,93.220
148,Afghanistan,2023,123.711
269,Africa,2020,564.834
...,...,...,...
22851,Yemen,2023,586.319
23065,Zambia,2020,135.259
23068,Zambia,2023,110.997
23190,Zimbabwe,2020,371.088


---
### Removing Null values from Carbon Intensity column

In [4]:
print("Null values in the Carbon Intensity column: ",df_carbon_intensity['carbon_intensity_elec'].isna().sum())

Null values in the Carbon Intensity column:  153


In [5]:
df_carbon_intensity = df_carbon_intensity.dropna(subset=['carbon_intensity_elec'])
print("Null values in the Carbon Intensity column: ", df_carbon_intensity['carbon_intensity_elec'].isna().sum())
display("OWID Carbon Intensity by Country",df_carbon_intensity.head())

Null values in the Carbon Intensity column:  0


'OWID Carbon Intensity by Country'

,country,year,carbon_intensity_elec
20,ASEAN (Ember),2020,589.474
23,ASEAN (Ember),2023,570.005
145,Afghanistan,2020,93.220
148,Afghanistan,2023,123.711
269,Africa,2020,564.834


---
### Removing Country Groups, Organizations and Continents from the column

In [6]:
keywords = [
    'Africa', 'Asia', 'Europe', 'America', 'Oceania', 'Union', 'OECD', 'OPEC',
    'income', 'World', 'Pacific', 'CIS', 'Shift', 'EI', 'EIA', 'Ember', 
    'Rest of World', 'G7', 'G20', 'Latin', 'Persian', 'Other', 
    'High-income', 'Low-income'
]

mask_remove_continents = ~df_carbon_intensity['country'].str.contains('|'.join(keywords), case=False, na=False)
df_carbon_intensity = df_carbon_intensity[mask_remove_continents]

---
## Q.1 Calculations for finding Global Carbon Emissions in MtCO₂ 
#### MtCO₂ = TWh × (10^9) × gCO₂/kWh × 10^(−12)

In [7]:
carbon_intensity_global2020 = df_carbon_intensity.loc[df_carbon_intensity['year'] == 2020, 'carbon_intensity_elec'].mean()
print("Mean value for 2020: ", carbon_intensity_global2020, " gCO₂/kWh")
carbon_intensity_global2023 = df_carbon_intensity.loc[df_carbon_intensity['year'] == 2023,'carbon_intensity_elec'].mean()
print("Mean value for 2023: ", carbon_intensity_global2023, " gCO₂/kWh")

Mean value for 2020:  454.4768221153846  gCO₂/kWh
Mean value for 2023:  424.1225176470589  gCO₂/kWh


In [8]:
twh_to_kwh = 1e9  # 1 TWh = 1e9 kWh
g_to_mt = 1e-12    # 1 g = 1e-12 Mt

def emissions_mt(twh, intensity):
    return twh * twh_to_kwh * intensity * g_to_mt  # MtCO₂

emissions_2020 = emissions_mt(df_iea.loc['Total','2020'], carbon_intensity_global2020)
emissions_2023 = emissions_mt(df_iea.loc['Total','2023'], carbon_intensity_global2023)
emissions_2024 = emissions_mt(df_iea.loc['Total','2024'], carbon_intensity_global2023)

emissions_2030_base = emissions_mt(df_iea.loc['Total','Base_2030'], carbon_intensity_global2023)
emissions_2035_base = emissions_mt(df_iea.loc['Total','Base_2035'], carbon_intensity_global2023)

emissions_2030_liftoff = emissions_mt(df_iea.loc['Total','LiftOff_2030'], carbon_intensity_global2023)
emissions_2035_liftoff = emissions_mt(df_iea.loc['Total','LiftOff_2035'], carbon_intensity_global2023)

emissions_2030_higheff = emissions_mt(df_iea.loc['Total','HighEff_2030'], carbon_intensity_global2023)
emissions_2035_higheff = emissions_mt(df_iea.loc['Total','HighEff_2035'], carbon_intensity_global2023)

emissions_2030_headwinds = emissions_mt(df_iea.loc['Total','Headwinds_2030'], carbon_intensity_global2023)
emissions_2035_headwinds = emissions_mt(df_iea.loc['Total','Headwinds_2035'], carbon_intensity_global2023)

print('''
emissions_2020: {:.2f} MtCO₂  | emissions_2030_base: {:.2f} MtCO₂      | emissions_2035_base: {:.2f} MtCO₂
emissions_2023: {:.2f} MtCO₂  | emissions_2030_liftoff: {:.2f} MtCO₂   | emissions_2035_liftoff: {:.2f} MtCO₂ 
emissions_2024: {:.2f} MtCO₂  | emissions_2030_higheff: {:.2f} MtCO₂   | emissions_2035_higheff: {:.2f} MtCO₂ 
----------------------------  | emissions_2030_headwinds: {:.2f} MtCO₂ | emissions_2035_headwinds: {:.2f} MtCO₂
'''.format(emissions_2020, emissions_2023, emissions_2024,
           emissions_2030_base, emissions_2035_base, 
           emissions_2030_liftoff, emissions_2035_liftoff,
           emissions_2030_higheff, emissions_2035_higheff,
           emissions_2030_headwinds, emissions_2035_headwinds))


emissions_2020: 122.25 MtCO₂  | emissions_2030_base: 153.11 MtCO₂      | emissions_2035_base: 176.43 MtCO₂
emissions_2023: 401.22 MtCO₂  | emissions_2030_liftoff: 505.98 MtCO₂   | emissions_2035_liftoff: 536.09 MtCO₂ 
emissions_2024: 729.07 MtCO₂  | emissions_2030_higheff: 335.91 MtCO₂   | emissions_2035_higheff: 412.25 MtCO₂ 
----------------------------  | emissions_2030_headwinds: 283.74 MtCO₂ | emissions_2035_headwinds: 299.85 MtCO₂



---
## Creating Dataframe (csv file) to import in Power BI

In [9]:
data = {
    'Scenario': [
        '2020', '2023', '2024',
        '2030_Base', '2035_Base',
        '2030_Liftoff', '2035_Liftoff',
        '2030_HighEff', '2035_HighEff',
        '2030_Headwinds', '2035_Headwinds'
    ],
    'TWh': [
        df_iea.loc['Total','2020'], df_iea.loc['Total','2023'], df_iea.loc['Total','2024'],
        df_iea.loc['Total','Base_2030'], df_iea.loc['Total','Base_2035'],
        df_iea.loc['Total','LiftOff_2030'], df_iea.loc['Total','LiftOff_2035'],
        df_iea.loc['Total','HighEff_2030'], df_iea.loc['Total','HighEff_2035'],
        df_iea.loc['Total','Headwinds_2030'], df_iea.loc['Total','Headwinds_2035']
    ]
}

Q1_AI_GlobalEmissions = pd.DataFrame(data)

# intensity column — use 2020’s for 2020, 2023’s for everything else
Q1_AI_GlobalEmissions['Intensity_gCO2_kWh'] = Q1_AI_GlobalEmissions['Scenario'].apply(
    lambda s: carbon_intensity_global2020 if s == '2020' else carbon_intensity_global2023
)

# emissions column using row-wise apply
Q1_AI_GlobalEmissions['Emissions_MtCO2'] = Q1_AI_GlobalEmissions.apply(
    lambda row: emissions_mt(row['TWh'], row['Intensity_gCO2_kWh']), axis=1
)

# baseline (2020 as baseline)
baseline_2020 = Q1_AI_GlobalEmissions.loc[Q1_AI_GlobalEmissions['Scenario'] == '2020', 'Emissions_MtCO2'].values[0]
# % change from baseline
Q1_AI_GlobalEmissions['%_Change_vs_2020'] = ((Q1_AI_GlobalEmissions['Emissions_MtCO2'] - baseline_2020) / baseline_2020) * 100

# dictionary mapping scenario to order number
order_map = {scenario: idx+1 for idx, scenario in enumerate(data['Scenario'])}
# SortOrder column
Q1_AI_GlobalEmissions['SortOrder'] = Q1_AI_GlobalEmissions['Scenario'].map(order_map)
# Reorder columns
Q1_AI_GlobalEmissions = Q1_AI_GlobalEmissions[['Scenario', 'SortOrder', 'TWh', 'Emissions_MtCO2', '%_Change_vs_2020']]

Q1_AI_GlobalEmissions

,Scenario,SortOrder,TWh,Emissions_MtCO2,%_Change_vs_2020
0,2020,1,269.0,122.254265,0.000000
1,2023,2,361.0,153.108229,25.237536
2,2024,3,416.0,176.434967,44.318047
3,2030_Base,4,946.0,401.219902,228.184789
4,2035_Base,5,1193.0,505.978164,313.873629
5,2030_Liftoff,6,1264.0,536.090862,338.504834
6,2035_Liftoff,7,1719.0,729.066608,496.352697
7,2030_HighEff,8,792.0,335.905034,174.759358
8,2035_HighEff,9,972.0,412.247087,237.204667
9,2030_Headwinds,10,669.0,283.737964,132.088397


In [46]:
# Converted to CSV for visualization on Power BI
Q1_AI_GlobalEmissions.to_csv("S:/PowerBI/Q1_AI_GlobalEmissions.csv", index=False)

---
### Dataframe for Stacked Area Chart in Power BI 

In [10]:
df_Reference_Scenarios = Q1_AI_GlobalEmissions.copy()

df_Reference_Scenarios['Year'] = df_Reference_Scenarios['Scenario'].str.extract(r'(\d{4})').astype(int)

branch_map = {
    'Base':'Base',
    'Liftoff':'Liftoff',
    'HighEff':'HighEff',
    'Headwinds':'Headwinds'
}

def get_branch(s):
    for k in branch_map.keys():
        if k.lower() in s.lower():
            return branch_map[k]
    return 'Base'

df_Reference_Scenarios['Branch'] = df_Reference_Scenarios['Scenario'].apply(get_branch)

In [11]:
df_Reference_Scenarios = df_Reference_Scenarios.pivot_table(index='Year', columns='Branch', values='Emissions_MtCO2')

df_Reference_Scenarios.reset_index(inplace=True)

df_Reference_Scenarios = df_Reference_Scenarios.fillna(0)

df_Reference_Scenarios

Branch,Year,Base,Headwinds,HighEff,Liftoff
0,2020,122.254265,0.000000,0.000000,0.000000
1,2023,153.108229,0.000000,0.000000,0.000000
2,2024,176.434967,0.000000,0.000000,0.000000
3,2030,401.219902,283.737964,335.905034,536.090862
4,2035,505.978164,299.854620,412.247087,729.066608


In [34]:
# Converted to CSV for visualization on Power BI
df_Reference_Scenarios.to_csv("S:/PowerBI/Reference_Scenarios.csv", index=False)

---
## Q2. Cloud Data Centers Selection impact
### Creating Dataframe containing only World Regions

In [12]:
columns_to_display = ['country','year','carbon_intensity_elec']
condition = (df_energy['year']==2023)
df_2023_regional = df_energy.loc[condition, columns_to_display]

df_2023_regional = df_2023_regional.dropna(subset=['carbon_intensity_elec'])

keywords = [
    'North America (Ember)', 'Europe (Ember)', 
    'Africa (Ember)', 'Middle East (Ember)',
    'Asia (Ember)', 'Oceania (Ember)',
    'Belize','Guatemala','Honduras','El Salvador','Nicaragua','Costa Rica','Panama',
    'South America'
]

mask = df_2023_regional['country'].isin(keywords)
df_2023_regional_cleaned = df_2023_regional[mask]

df_2023_regional_cleaned

,country,year,carbon_intensity_elec
401,Africa (Ember),2023,541.355
1414,Asia (Ember),2023,590.394
2553,Belize,2023,155.556
4953,Costa Rica,2023,24.765
6476,El Salvador,2023,118.462
7189,Europe (Ember),2023,300.277
8887,Guatemala,2023,272.660
9248,Honduras,2023,289.496
12569,Middle East (Ember),2023,641.222
13885,Nicaragua,2023,288.330


---
### Calculating Carbon Intensity for Asia Pacific region

In [13]:
asia_oceania_mean = df_2023_regional_cleaned.loc[
    df_2023_regional_cleaned['country'].isin(['Asia (Ember)','Oceania (Ember)']),
    'carbon_intensity_elec'].mean()

new_row_asia_pacific = {
    'country': 'Asia Pacific (Calculated)',
    'year': 2023,
    'carbon_intensity_elec': asia_oceania_mean
}

df_2023_regional_cleaned = pd.concat(
    [df_2023_regional_cleaned, pd.DataFrame([new_row_asia_pacific])],
    ignore_index = True
)

df_2023_regional_cleaned.iloc[14:]

,country,year,carbon_intensity_elec
14,Asia Pacific (Calculated),2023,542.5765


---
### Calculating Carbon Intensity for Central & South America region

In [14]:
central_america_mean = df_2023_regional_cleaned.loc[
    df_2023_regional_cleaned['country'].isin(
        ['Belize','Guatemala','Honduras','El Salvador','Nicaragua','Costa Rica','Panama']),
    'carbon_intensity_elec'].mean()

south_america_intensity = df_2023_regional_cleaned.loc[
    df_2023_regional_cleaned['country'] == 'South America',
    'carbon_intensity_elec'].values[0]

central_south_mean = (central_america_mean + south_america_intensity)/2

new_row_central_south = {
    'country': 'Central and South America (Calculated)',
    'year': 2023,
    'carbon_intensity_elec': central_south_mean
}

df_2023_regional_cleaned = pd.concat(
    [df_2023_regional_cleaned, pd.DataFrame([new_row_central_south])],
    ignore_index = True
)

df_2023_regional_cleaned.iloc[15:]

,country,year,carbon_intensity_elec
15,Central and South America (Calculated),2023,186.405643


---
### Calculating Carbon Emissions for all Regions

In [15]:
NA_emissions_2023 = emissions_mt(158, 363.316)
CSA_emissions_2023 = emissions_mt(1.5, 186.405643)
Europe_emissions_2023 = emissions_mt(66, 300.277000)
Africa_emissions_2023 = emissions_mt(1.3, 541.355000)
MidEast_emissions_2023 = emissions_mt(1.3, 641.222000)
AsiaPacific_emissions_2023 = emissions_mt(128, 542.576500)

data = {
    'Region': [
        'North America', 'Central and South America',
        'Europe','Africa','Middle East','Asia Pacific'
    ],
    'TWh': [ 158, 1.5, 66, 1.3, 1.3, 128],
    'Carbon_Intensity_gCO2kWh':[
        363.316, 186.405643, 300.277000, 541.355000, 641.222000, 542.576500],
    'Emissions_MtCO2': [
        57.40, 0.28, 19.82, 0.70, 0.83, 69.45]
}
# build dataframe
df_2023_regional_cleaned = pd.DataFrame(data)
df_2023_regional_cleaned

,Region,TWh,Carbon_Intensity_gCO2kWh,Emissions_MtCO2
0,North America,158.0,363.316000,57.40
1,Central and South America,1.5,186.405643,0.28
2,Europe,66.0,300.277000,19.82
3,Africa,1.3,541.355000,0.70
4,Middle East,1.3,641.222000,0.83
5,Asia Pacific,128.0,542.576500,69.45


In [72]:
# Converted to CSV for visualization on Power BI
df_2023_regional_cleaned.to_csv("S:/PowerBI/Regional_Emissions.csv", index=False)

---
### Creating a separate dataframe for Filled Map in Power BI

In [16]:
region_intensity = {
    'North America': 363.316,
    'Central and South America': 186.405643,
    'Europe': 300.277000,
    'Africa': 541.355000,
    'Middle East': 641.222000,
    'Asia Pacific': 542.576500
}

north_america = ['Canada', 'Mexico','United States']
central_south = ['Argentina','Bolivia','Brazil','Chile','Colombia','Costa Rica','Cuba','Curacao','Dominican Republic',
                 'Ecuador','El Salvador','Guatemala','Haiti','Honduras','Jamaica','Nicaragua','Panama','Paraguay',
                 'Peru','Suriname','Trinidad and Tobago','Uruguay','Venezuela']
europe = ['Albania','Austria','Belarus','Belgium','Bosnia and Herzegovina','Bulgaria','Croatia','Cyprus','Czechia',
          'Denmark','Estonia','Finland','France','Germany','Gibraltar','Greece','Hungary','Iceland','Ireland','Israel',
          'Italy','Kosovo','Latvia','Lithuania','Luxembourg','Malta','Moldova','Montenegro','North Macedonia','Norway',
          'Poland','Portugal','Romania','Serbia','Slovak Republic','Slovenia','Spain','Sweden','Switzerland','Netherlands',
          'Türkiye','Ukraine','United Kingdom']
africa = ['Algeria','Angola','Benin','Botswana','Burkina Faso','Burundi','Cabo Verde','Cameroon','Central African Republic',
          'Chad','Comoros','Congo','Cote D’Ivoire','Democratic Republic of the Congo','Djibouti','Egypt','Equatorial Guinea',
          'Eritrea','Eswatini','Ethiopia','Gabon','Gambia','Ghana','Guinea','Guinea-Bissau','Kenya','Lesotho','Liberia','Libya',
          'Madagascar','Malawi','Mali','Mauritania','Mauritius','Morocco','Mozambique','Namibia','Niger','Nigeria','Rwanda',
          'Sao Tome and Principe','Senegal','Seychelles','Sierra Leone','Somalia','South Africa','South Sudan','Sudan','Tanzania',
          'Togo','Tunisia','Uganda','Zambia','Zimbabwe']
middle_east = ['Bahrain','Iran','Iraq','Jordan','Kuwait','Lebanon','Oman','Qatar','Saudi Arabia','Syria','United Arab Emirates','Yemen']
asia_pacific = ['Australia','Bangladesh','Brunei','Cambodia','China','Hong Kong','India','Indonesia','Japan','Korea',
                'Laos','Malaysia','Mongolia','Myanmar','Nepal','New Zealand','Pakistan','Philippines','Singapore',
                'Sri Lanka','Taiwan','Thailand','Vietnam']

rows = []
for c in north_america:
    rows.append([c,region_intensity['North America']])
for c in central_south:
    rows.append([c,region_intensity['Central and South America']])
for c in europe:
    rows.append([c,region_intensity['Europe']])
for c in africa:
    rows.append([c,region_intensity['Africa']])
for c in middle_east:
    rows.append([c,region_intensity['Middle East']])
for c in asia_pacific:
    rows.append([c,region_intensity['Asia Pacific']])

df_map = pd.DataFrame(rows, columns=['Country','Carbon_Intensity_gCO2kWh'])

df_map.to_csv("S:/PowerBI/CarbonIntensity_Map.csv", index=False)

print(df_map.head(), "\nTotal countries:", len(df_map))

         Country  Carbon_Intensity_gCO2kWh
0         Canada                363.316000
1         Mexico                363.316000
2  United States                363.316000
3      Argentina                186.405643
4        Bolivia                186.405643 
Total countries: 158


In [17]:
df_map

,Country,Carbon_Intensity_gCO2kWh
0,Canada,363.316000
1,Mexico,363.316000
2,United States,363.316000
3,Argentina,186.405643
4,Bolivia,186.405643
...,...,...
153,Singapore,542.576500
154,Sri Lanka,542.576500
155,Taiwan,542.576500
156,Thailand,542.576500


---
## Q3. Calculations for Shifted Scenario (20% of TWh from Asia-Pacific to Europe)

In [18]:
# Current data
data_current = {
    'Region': [
        'North America', 'Central and South America',
        'Europe','Africa','Middle East','Asia Pacific'
    ],
    'TWh': [158, 1.5, 66, 1.3, 1.3, 128],
    'Intensity': [363.316, 186.405643, 300.277000, 541.355000, 641.222000, 542.576500]
}
df_current = pd.DataFrame(data_current)
df_current['Emissions_MtCO2'] = emissions_mt(df_current['TWh'], df_current['Intensity'])

# Shift scenario: move 20% Asia Pacific to Europe
df_shift = df_current.copy()
df_shift.loc[df_shift['Region']=='Asia Pacific','TWh'] = 128*0.8
df_shift.loc[df_shift['Region']=='Europe','TWh'] = 66 + (128*0.2)

df_shift['Emissions_MtCO2'] = emissions_mt(df_shift['TWh'], df_shift['Intensity'])

# Combine scenarios
df_current['Scenario'] = 'Current'
df_shift['Scenario'] = 'Shifted'

df_scenarios = pd.concat([df_current, df_shift])
df_scenarios = df_scenarios[['Scenario','Region','TWh','Intensity','Emissions_MtCO2']]

print(df_scenarios)

  Scenario                     Region    TWh   Intensity  Emissions_MtCO2
0  Current              North America  158.0  363.316000        57.403928
1  Current  Central and South America    1.5  186.405643         0.279608
2  Current                     Europe   66.0  300.277000        19.818282
3  Current                     Africa    1.3  541.355000         0.703762
4  Current                Middle East    1.3  641.222000         0.833589
5  Current               Asia Pacific  128.0  542.576500        69.449792
0  Shifted              North America  158.0  363.316000        57.403928
1  Shifted  Central and South America    1.5  186.405643         0.279608
2  Shifted                     Europe   91.6  300.277000        27.505373
3  Shifted                     Africa    1.3  541.355000         0.703762
4  Shifted                Middle East    1.3  641.222000         0.833589
5  Shifted               Asia Pacific  102.4  542.576500        55.559834


In [11]:
# Converted to CSV for visualization on Power BI
df_scenarios.to_csv("S:/PowerBI/Shifted_Scenario.csv", index=False)

---
### Dataframe for Waterfall Chart in Power BI

In [19]:
import pandas as pd

waterfall_data = pd.DataFrame({
    'Step': [
        'Current Total',
        'Asia Pacific Reduction',
        'Europe Increase',
        'Shifted Total'
    ],
    'Value': [
        148.488961,  # total now
        -13.889958,   # reduction
        +7.687091,   # increase
        142.286094   # new total
    ],
    'Category': [
        'Total','Decrease','Increase','Total'
    ]
})
waterfall_data['SortOrder'] = [1,2,3,4] 
print(waterfall_data)


                     Step       Value  Category  SortOrder
0           Current Total  148.488961     Total          1
1  Asia Pacific Reduction  -13.889958  Decrease          2
2         Europe Increase    7.687091  Increase          3
3           Shifted Total  142.286094     Total          4


In [17]:
# Converted to CSV for visualization on Power BI
waterfall_data.to_csv("S:/PowerBI/Waterfall_Emissions.csv", index=False)